#Tolkien Elvish Translator#

The Keras blog has a post that implements a simple Encoder and Decoder to tranlate English-French Phrases



All Data scraped from: http://eldamo.org/


In [9]:
import numpy as np
import pandas as pd
import keras

def raw_data_reader(raw_file_path,out_path):
    dat=open(raw_file_path,"r",encoding="utf-8")
    out=open(out_path,"w",encoding="utf-8")
    for i in dat:
        try:
            tmp=i.split("“")
        except:
            print ("****BAD***LINE***********")
        tmp[0]=tmp[0].strip(",”*[]†²\n\t{}><·’;")
        tmp[1]=tmp[1].strip(",”*[]†²\n\t{}><·’;")
        out.write(tmp[1]+",\t"+tmp[0]+"\n")
    out.close()

def encodeData(csvFilePath):
    inSeq=[]#Input phrases/sequences
    outSeq=[]
    inChars=set()#Characters used in input
    outChars=set()
    
    csvFile=open(csvFilePath,"r",encoding="utf-8")
    for i in csvFile:
        tmp=(i.split(","))
        inSeq.append(tmp[0])#Append sentences to list
        outSeq.append(tmp[1])#Elvish phrases list
        #Add to the list of English Characters
        for j in tmp[0]:
            for tmpChar in j:
                if tmpChar not in inChars:
                    inChars.add(tmpChar)
        #...significantly more elvish Characters            
        for j in tmp[1]:
            for tmpChar in j:
                if tmpChar not in outChars:
                    outChars.add(tmpChar)
                    
    return inSeq,outSeq,sorted(list(inChars)),sorted(list(outChars))

In [10]:
process_raw=True
if process_raw:
    inPath,outPath="raw_data/","processed_data/"
    raw_data_reader(inPath+"Sindarin_Phrases.dat",outPath+"Sindarin_Phrases.csv")
    raw_data_reader(inPath+"Quenya_Phrases.dat",outPath+"Quenya_Phrases.csv")

quenya=True
if quenya:
    print ("Loading the Quenya Phrase dataset")
    input_seq,out_seq,input_chars,out_chars=encodeData(outPath+"Quenya_Phrases.csv")
    modelPath="models/Quenya_LSTM.h5"
else:
    input_seq,out_seq,input_chars,out_chars=encodeData(outPath+"Sindarin_Phrases.csv") 
    modelPath="models/Sindarin_LSTM.h5"

Loading the Quenya Phrase dataset


In [11]:
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(out_chars)
max_encoder_seq_length = max([len(txt) for txt in input_seq])
max_decoder_seq_length = max([len(txt) for txt in out_seq])

print('Number of samples:', len(input_seq))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 749
Number of unique input tokens: 78
Number of unique output tokens: 84
Max sequence length for inputs: 128
Max sequence length for outputs: 121


In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(out_chars)])

encodedInArray=np.zeros((len(input_seq), max_encoder_seq_length,\
                         num_encoder_tokens),dtype='float32')
decodedInArray=np.zeros((len(input_seq), max_decoder_seq_length,\
                         num_decoder_tokens),dtype='float32')
decodedOutArray=np.zeros((len(input_seq), max_decoder_seq_length,\
                          num_decoder_tokens),dtype='float32')

In [13]:
print ("Input Token Index")
print (input_token_index)

print ("")
print ("Target Token Index")
print (target_token_index)

Input Token Index
{' ': 0, '!': 1, '(': 2, ')': 3, '*': 4, '-': 5, '.': 6, '2': 7, '3': 8, ':': 9, ';': 10, '=': 11, '>': 12, '?': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, 'H': 21, 'I': 22, 'J': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'O': 28, 'P': 29, 'Q': 30, 'R': 31, 'S': 32, 'T': 33, 'V': 34, 'W': 35, '[': 36, ']': 37, 'a': 38, 'b': 39, 'c': 40, 'd': 41, 'e': 42, 'f': 43, 'g': 44, 'h': 45, 'i': 46, 'j': 47, 'k': 48, 'l': 49, 'm': 50, 'n': 51, 'o': 52, 'p': 53, 'q': 54, 'r': 55, 's': 56, 't': 57, 'u': 58, 'v': 59, 'w': 60, 'x': 61, 'y': 62, 'z': 63, '{': 64, '}': 65, 'Í': 66, 'ä': 67, 'é': 68, 'ë': 69, 'í': 70, 'ð': 71, 'ó': 72, 'ú': 73, '–': 74, '—': 75, '‘': 76, '’': 77}

Target Token Index
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '(': 4, ')': 5, '-': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'B': 12, 'C': 13, 'E': 14, 'F': 15, 'G': 16, 'H': 17, 'I': 18, 'K': 19, 'L': 20, 'M': 21, 'N': 22, 'O': 23, 'P': 24, 'Q': 25, 'R': 26, 'S': 27, 'T': 28, 'V': 29, 'W': 3

In [14]:
for i, (input_text, target_text) in enumerate(zip(input_seq, out_seq)):
    print (input_text,"|",target_text.strip("\n"))
    for t, char in enumerate(input_text):
        encodedInArray[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decodedInArray[i, t, target_token_index[char]] = 1.  # Target data is offset by one
        if t > 0:
            decodedOutArray[i, t - 1, target_token_index[char]] = 1.

upon the shining shore | 	ailinisse alkarain 
upon the shining shore | 	ailinissen alkarain 
on the last shores | 	ailinissen oilimaisen 
upon the last beaches | 	ailissen oilimaisen 
the old darkness beyond the stars | 	aire móre ala tinwi 
it shone like gold | 	a kálie kulundon 
after the last night | 	ala fuin oilimaite 
after the last night | 	ala fuin oilimaite² 
before the last night | 	ala hui oilimaite 
before the last night | 	ala hui oilimaite² 
before the last night | 	ala hui oilimaite³ 
made it shine in the lights of the sun | 	alkantaniéren úrio kalmainen 
in the last rays of light | 	alkarissen oilimain 
in the last rays of light | 	alkarissen oilimain² 
(a) long wing | 	anda ráma 
give (it) me | 	an ni 
give (it) me | 	an nir 
went on the heights driven by the wind a ship like a bird with a blossom-white neck | 	anwe or aiqale elta súrut lunte aiwendon lossiattea 
and the sails of the ship will shine with golden lights | 	ar i·kiryo kaluváre talain kulukalmalínen 
and t

glory be to the Father and to the Son and to the Holy Spirit | 	alcar i Ataren ar i Yondon ar i Airefean 
O glorious and blessed Virgin | 	alcarin Vendë ar manaquenta 
glory [be] to God in the highest | 	alcar mi Tarmenel na Erun 
the splendour of Oromë | 	alcar Oromëo 
the splendour of Oromë | 	alcar Oroméva 
Galadriel’s lament in Lórien | 	Altariello nainië Lóriendesse 
give us this day our daily bread | 	ámen anta síra ilaurëa massamma 
Mother of divine grace | 	Amillë Eruva lissëo 
Mother of Christ | 	Amillë Hristo 
A is brighter than B; (lit.) A is bright beyond B | 	A (ná) calima lá B 
A is dear before B | 	A anameldë na epë B 
A is brightest of all | 	A ancalima imb’ illi 
A is (very much) brighter than B | 	A arcalima (ná) epe B 
brighter than stars | 	ancalima ep’ eleni 
brightest of stars | 	elenion ancalima 
be well | 	á na márië 
The sun shall shine upon your path | 	Anar caluva tielyanna 
a bleared sun blinking | 	anar púrëa tihta 
long will we praise them | 	andavë laituv

The Natures and Visible Shapes of the Valar and Maiar | 	Nasser ar Cenime Cantar Valaron ar Maiaron 
he will try to come | 	nauva túlë 
air-breathing animals | 	nefíti kuimar 
he was a horse strong swift at running | 	nésë nórima rocco 
the white rocks snarling | 	ninqui carcar yarra 
Ingoldo’s History of the Noldor | 	Noldo-quentasta Ingoldova 
he ran quickly | 	nornë lintië 
he ran as quickly as he could (lit.) he ran with his speed | 	nornë lintieryanen 
tripping lightly whirling lightly | 	norolinda pirucendëa 
under ruined skies | 	nu fanyarë rúcina 
The One’s perpetual production | 	Oiencarmë Eruo 
in the song of her voice holy and queenly | 	ómaryo airetári-lírinen 
in [by means of] her voice’s song of the holy-queen | 	ómaryo lírinen airetário 
by the song of the voice of the holy queen | 	lírinen ómo i·aire táríva 
on the dark rocks | 	ondolissë mornë 
Enquiry into the Place-names of Gondor | 	Ondonórë Nómesseron Minasurië 
but deliver us always from all dangers | 	ono alyë et

In [15]:
print ("What do the encodings look like?")
print (encodedInArray[0])
print("")

What do the encodings look like?
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]



In [16]:
from keras.models import Model,load_model
from keras.layers import Input, LSTM, Dense

print ("Building the model:")
print ("Copied directly from Keras Blog")
batch_size = 64  # Batch size for training.
epochs = 30 # Number of epochs to train for.
latent_dim = 1024  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Building the model:
Copied directly from Keras Blog


In [17]:


train=True
if not train:
    model = load_model(modelPath)
else:
    # Run training
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    model.fit([encodedInArray, decodedInArray], decodedOutArray,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.1)

    # Save model
    model.save(modelPath)

Train on 674 samples, validate on 75 samples
Epoch 1/30
674/674 [==============================] - 130s 192ms/step - loss: 0.7711 - val_loss: 0.6747
Epoch 2/30
674/674 [==============================] - 131s 195ms/step - loss: 0.5806 - val_loss: 0.6130
Epoch 3/30
674/674 [==============================] - 133s 197ms/step - loss: 0.5629 - val_loss: 0.6136
Epoch 4/30
674/674 [==============================] - 127s 189ms/step - loss: 0.5599 - val_loss: 0.6051
Epoch 5/30
674/674 [==============================] - 129s 191ms/step - loss: 0.5565 - val_loss: 0.5958
Epoch 6/30
674/674 [==============================] - 128s 190ms/step - loss: 0.5479 - val_loss: 0.6029
Epoch 7/30
674/674 [==============================] - 129s 191ms/step - loss: 0.5365 - val_loss: 0.6095
Epoch 8/30
674/674 [==============================] - 130s 193ms/step - loss: 0.5149 - val_loss: 0.5563
Epoch 9/30
674/674 [==============================] - 130s 193ms/step - loss: 0.5016 - val_loss: 0.5341
Epoch 10/30
674/674

C:\Users\j\Anaconda3\lib\site-packages\keras\engine\topology.py:2361: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 1024) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 1024) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [18]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [1]:
print ("Reverse input token lookup:")
print (reverse_input_char_index)
print ("")
print ("Reverse target token lookup:")
print (reverse_target_char_index)

Reverse input token lookup:


NameError: name 'reverse_input_char_index' is not defined

In [19]:
#Build encoder and decoder system

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [20]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [21]:
#Investigate the performance.
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    inputSeq = encodedInArray[seq_index: seq_index + 1]
    #print (inputSeq)
    decoded_sentence = decode_sequence(inputSeq)
    print('-')
    print('Input sentence:', input_seq[seq_index])
    print('Decoded sentence:', decoded_sentence.strip("\n"))
    print("Target sentence: ",out_seq[seq_index])

-
Input sentence: upon the shining shore
Decoded sentence: man kiluva nar 
Target sentence:  	ailinisse alkarain 

-
Input sentence: upon the shining shore
Decoded sentence: man kiluva nar 
Target sentence:  	ailinissen alkarain 

-
Input sentence: on the last shores
Decoded sentence: man kiluva nar 
Target sentence:  	ailinissen oilimaisen 

-
Input sentence: upon the last beaches
Decoded sentence: man kiluva nar 
Target sentence:  	ailissen oilimaisen 

-
Input sentence: the old darkness beyond the stars
Decoded sentence: man kiluva nar 
Target sentence:  	aire móre ala tinwi 

-
Input sentence: it shone like gold
Decoded sentence: man kiluva nar 
Target sentence:  	a kálie kulundon 

-
Input sentence: after the last night
Decoded sentence: man kiluva nar 
Target sentence:  	ala fuin oilimaite 

-
Input sentence: after the last night
Decoded sentence: man kiluva nar 
Target sentence:  	ala fuin oilimaite² 

-
Input sentence: before the last night
Decoded sentence: man kiluva nar 
Tar

-
Input sentence: the white ship lay upon the rocks
Decoded sentence: man kiluva nar 
Target sentence:  	kaire laiqa’ondoisen kirya 

-
Input sentence: as a corpse into the grave
Decoded sentence: man kiluva nar 
Target sentence:  	kaivo i sapsanta 

-
Input sentence: the gleaming-moon goes down like a corpse into the grave
Decoded sentence: man kiluva nar 
Target sentence:  	kaivon nyúken i·sapsanta silmerána númetár 

-
Input sentence: the light fading
Decoded sentence: man kiluva nar 
Target sentence:  	kalma histane 

-
Input sentence: amid the red skies the Sun with wet eyes dropped tears of mist
Decoded sentence: man kiluva nar 
Target sentence:  	karnevaite úri kilde hísen níe nienaite 

-
Input sentence: red-skied the sun will gaze through a haze of tears
Decoded sentence: man kiluva nar 
Target sentence:  	karnevaite úri kilivande hísen nie nie nienaite 

-
Input sentence: a deed of (some actual) war
Decoded sentence: man kiluva nar 
Target sentence:  	karo ohtan 

-
Input sen